In [ ]:
from audioop import reverse
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.aer import AerSimulator, AerProvider
from qiskit import IBMQ
from qiskit.compiler import transpile
from time import perf_counter
import numpy as np
from qiskit.circuit.library.standard_gates import SGate, SdgGate
from ctypes import BigEndianStructure
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit import Aer

matrix = np.array([[2/np.sqrt(5), 1/np.sqrt(5)],[1/np.sqrt(5), 2/np.sqrt(5)]])
sol = np.array([np.sqrt(3)/2, 1/2])

qubits = QuantumRegister(4)
measurement = ClassicalRegister(4)
circuit = QuantumCircuit(qubits,measurement)

#t = number of variables + log(2+1/(2*error)) //some magic number
t = 2.344916

#r = number that increases precision of solution as it gets higher, but requires more computation
r = 2

#load sol
circuit.ry(2*np.arccos(matrix[0][0]),qubits[-1])

last_index = len(qubits)-1

In [ ]:
#phase estimation

#set superposition
circuit.h(qubits[1])
circuit.h(qubits[2])

#trotterization
beta = matrix[0][1]
gamma = (matrix[0][0]-matrix[1][1])/2
iterations = 2**(r-1)

for i in (range(1,last_index)):
    for x in range(iterations):
        circuit.crz(((-2*t*gamma*2**(i-1))/r),qubits[last_index-i],qubits[-1])
        circuit.crx(((-2*t*beta*2**(i-1))/r),qubits[last_index-i],qubits[-1])


circuit.swap(qubits[1],qubits[2])
circuit.h(qubits[2])

csdg = SdgGate().control(1) #create a controlled adjoint S gate
circuit.append(csdg,[1,2])


circuit.h(qubits[1])

#controlled rotation
circuit.swap(qubits[1],qubits[2])
circuit.cry((2*np.pi)/(2**r),qubits[1],qubits[0])
circuit.cry((np.pi)/(2**r),qubits[2],qubits[0])
circuit.swap(qubits[1],qubits[2])

#inverse phase estimation

circuit.h(qubits[1])

cs = SGate().control(1) #create a controlled S gate
circuit.append(cs,[1,2])

circuit.h(qubits[2])
circuit.swap(qubits[1],qubits[2])

#reverse trotterization
for i in reversed(range(1,last_index)):
    for x in range(iterations):
        circuit.crx(((2*t*beta*2**(i-1))/r),qubits[last_index-i],qubits[-1])
        circuit.crz(((2*t*gamma*2**(i-1))/r),qubits[last_index-i],qubits[-1])
        
#undo superposition
circuit.h(qubits[1])
circuit.h(qubits[2])
    
circuit.measure(qubits,measurement)

print(circuit)


simulator = Aer.get_backend('aer_simulator')
simulation = execute(circuit, simulator, shots=1000)
result = simulation.result()

counts = result.get_counts(circuit)
for(measured_state, count) in counts.items():

# Qiskit will give you the state in little-endian form by default, so if you want to
# turn it into big-endian, just reverse the state
     big_endian_state = measured_state[::-1]
     print(f"Measured {big_endian_state} {count} times.")

#Only count instances when last qubit is 1